# Project


In [1]:
import Pkg
using JuMP
using Cbc
using DelimitedFiles
using NBInclude

# constante
const ϵ = 0.00001

SyntaxError: ignored

In [ ]:
using JuMP
#import Pkg
#using Gurobi
using Cbc
#Pkg.add("NBInclude")
using DelimitedFiles
using NBInclude
#const GUROBI_ENV = Gurobi.Env()
const ϵ = 0.00001

1.0e-5

In [ ]:
#@nbinclude("master.ipynb")
#@nbinclude("subproblem.ipynb")
#@nbinclude("branch_and_price.ipynb")
#@nbinclude("node.ipynb")

solve_MIP (generic function with 1 method)

In [ ]:
global D = read_data("Example.txt")

********** Read instance Example.txt ****************
Number of machines is: 3
Number of jobs is: 5
Machine capacities are: [28, 26, 18]. This is a 1xm array.
Assignment costs are: [9 19 4 29 9; 1 24 14 4 24; 24 1 34 4 14]. This is an mxn array.
Processing times are: [11 7 22 9 15; 12 16 20 10 10; 9 20 7 23 5]. This is an mxn array.


0
1
2
3
4
